# Parse Season and Episode Info

---

## Read in season information

In [22]:
INPUT_FILE = 'season_info.txt'

with open(INPUT_FILE, 'r') as f:
    lines = [line.strip() for line in f.readlines()]
# lines
    
# Remove all blank lines
lines = [line for line in lines if line != '']

---

## Classes for Season and Episode

In [2]:
class Season:
    def __init__(self, season_no: int, air_dates: list[str]):
        self.season_no = season_no
        self.air_dates = air_dates
        self.episodes = []
    
    def add_episode(self, episode):
        self.episodes.append(episode)

---

### Create seasons

In [24]:
# Extracts only the lines containing the season info
season_lines = [line for line in lines if 'Season' in line]

In [23]:
# Saves season info as dicts
season_info_json = []

# Check season lines for season number and air dates
for season_line in season_lines:
    # Dict to hold individual season info
    s = {}

    # Split the line to extract required data
    season_info = season_line.split()

    # SEASON NO
    season_no = season_info[1]
    
    # AIR DATES
    air_dates = []
    date = season_info[-1][1:-1]
    if '-' not in date:
        air_dates.append(date)
    else:
        date = date.split('-')
        start_date = f'19{date[0]}'
        end_date = f'19{date[1]}'
        air_dates = [start_date, end_date]
    
    # Create dict
    s['season_id'] = 
    s['season_no'] = season_no
    s['air_dates'] = air_dates
    s['episodes'] = []
    # Add dict to list
    season_info_json.append(s)

# for s in season_info_json:
#     print(s)

In [4]:
# # Class objects for seasons

# season_1 = Season(season_info_json[0]['season_no'], season_info_json[0]['air_dates'])
# season_2 = Season(season_info_json[1]['season_no'], season_info_json[1]['air_dates'])
# season_3 = Season(season_info_json[2]['season_no'], season_info_json[2]['air_dates'])
# season_4 = Season(season_info_json[3]['season_no'], season_info_json[3]['air_dates'])
# season_5 = Season(season_info_json[4]['season_no'], season_info_json[4]['air_dates'])
# season_6 = Season(season_info_json[5]['season_no'], season_info_json[5]['air_dates'])
# season_7 = Season(season_info_json[6]['season_no'], season_info_json[6]['air_dates'])
# season_8 = Season(season_info_json[7]['season_no'], season_info_json[7]['air_dates'])
# season_9 = Season(season_info_json[8]['season_no'], season_info_json[8]['air_dates'])

# print(season_1.air_dates)

['1989']


---

### Create Episodes

In [5]:
# Separate lines into sections by season
season_info = []
# Holds the line number of each line containing the season info
season_indices = []

# Find the lines with the word 'season' in them
for i, line in enumerate(lines):
    if 'Season' in line:
        season_indices.append(i)

# Create a list of lines for each season
for i in range(len(season_indices)-1):
    j = season_indices[i]
    k = season_indices[i+1]
    current_season = lines[j:k]
    season_info.append(current_season)

# Don't forget the last season's lines
last_season = lines[k:]
season_info.append(last_season)

# season_info[:10]

[['Season 1 (1989)',
  '1\tGood News Bad News (7/5/89)',
  '2\tThe Stakeout (5/31/90)',
  '3\tThe Robbery (6/7/90)',
  '4\tMale Unbonding (6/14/90)',
  '5\tThe Stock Tip (6/21/90)'],
 ['Season 2 (1991)',
  '6\tThe Ex-Girlfriend (1/23/91)',
  '7\tThe Pony Remark (1/30/91)',
  '8\tThe Jacket (2/6/91)',
  '9\tThe Phone Message (2/13/91)',
  '10\tThe Apartment (4/4/91)',
  '11\tThe Statue (4/11/91)',
  '12\tThe Revenge (4/18/91)',
  '13\tThe Heart Attack (4/25/91)',
  '14\tThe Deal (5/2/91)',
  '15\tThe Baby Shower (5/16/91)',
  '16\tThe Chinese Restaurant (5/23/91)',
  '17\tThe Busboy (6/26/91)'],
 ['Season 3 (91-92)',
  '18\tThe Note (9/18/91)',
  '19\tThe Truth (9/25/91)',
  '20\tThe Pen (10/2/91)',
  '21\tThe Dog (10/9/91)',
  '22\tThe Library (10/16/91)',
  '23\tThe Parking Garage (10/30/91)',
  '24\tThe Cafe (11/6/91)',
  '25\tThe Tape (11/13/91)',
  '26\tThe Nose Job (11/20/91)',
  '27\tThe Stranded (11/27/91)',
  '28\tThe Alternate Side (12/4/91)',
  '29\tThe Red Dot (12/11/91)',
 

In [9]:
episode_info_json = []

# For each season 
for i, s in enumerate(season_info, 1):
    
    season_no = i
    
    # The first line is the 'Season n' line
    episode_info = s[1:]
    # Episode info
    for j, episode in enumerate(episode_info, 1):
        
        # EPISODE NO
        episode_no = j
        
        # Split the line into parts
        info = episode.split()
        
        # CHRONO NO
        chrono_no = info[0]

        # AIR DATE
        air_date = info[-1][1:-1].split('/')
        # Holds formatted date sections
        new_date = []
        # Date sections
        month = air_date[0]
        day = air_date[1]
        year = air_date[2]
        # Make each date section formatted correctly
        if len(month) != 2:
            month = f'0{month}'
        if len(day) != 2:
            day = f'0{day}'
        if len(year) != 4:
            year = f'19{year}'
        new_date = [month, day, year]
        # Combine date parts
        air_date = '/'.join(new_date)

        # TITLE
        title = ' '.join(info[1:-1])

        # JSON
        ep_info = {}
        ep_info['season_no'] = season_no
        ep_info['episode_no'] = episode_no
        ep_info['title'] = title
        ep_info['air_date'] = air_date
        ep_info['chrono_no'] = chrono_no
        ep_info['quotes'] = []
        episode_info_json.append(ep_info)

for e in episode_info_json:
    print(e)


{'season_no': 1, 'episode_no': 1, 'title': 'Good News Bad News', 'air_date': '07/05/1989', 'chrono_no': '1', 'quotes': []}
{'season_no': 1, 'episode_no': 2, 'title': 'The Stakeout', 'air_date': '05/31/1990', 'chrono_no': '2', 'quotes': []}
{'season_no': 1, 'episode_no': 3, 'title': 'The Robbery', 'air_date': '06/07/1990', 'chrono_no': '3', 'quotes': []}
{'season_no': 1, 'episode_no': 4, 'title': 'Male Unbonding', 'air_date': '06/14/1990', 'chrono_no': '4', 'quotes': []}
{'season_no': 1, 'episode_no': 5, 'title': 'The Stock Tip', 'air_date': '06/21/1990', 'chrono_no': '5', 'quotes': []}
{'season_no': 2, 'episode_no': 1, 'title': 'The Ex-Girlfriend', 'air_date': '01/23/1991', 'chrono_no': '6', 'quotes': []}
{'season_no': 2, 'episode_no': 2, 'title': 'The Pony Remark', 'air_date': '01/30/1991', 'chrono_no': '7', 'quotes': []}
{'season_no': 2, 'episode_no': 3, 'title': 'The Jacket', 'air_date': '02/06/1991', 'chrono_no': '8', 'quotes': []}
{'season_no': 2, 'episode_no': 4, 'title': 'The Ph

[{'season_no': 1,
  'episode_no': 1,
  'title': 'Good News Bad News',
  'air_date': '07/05/1989',
  'chrono_no': '1',
  'quotes': []},
 {'season_no': 1,
  'episode_no': 2,
  'title': 'The Stakeout',
  'air_date': '05/31/1990',
  'chrono_no': '2',
  'quotes': []},
 {'season_no': 1,
  'episode_no': 3,
  'title': 'The Robbery',
  'air_date': '06/07/1990',
  'chrono_no': '3',
  'quotes': []},
 {'season_no': 1,
  'episode_no': 4,
  'title': 'Male Unbonding',
  'air_date': '06/14/1990',
  'chrono_no': '4',
  'quotes': []},
 {'season_no': 1,
  'episode_no': 5,
  'title': 'The Stock Tip',
  'air_date': '06/21/1990',
  'chrono_no': '5',
  'quotes': []},
 {'season_no': 2,
  'episode_no': 1,
  'title': 'The Ex-Girlfriend',
  'air_date': '01/23/1991',
  'chrono_no': '6',
  'quotes': []},
 {'season_no': 2,
  'episode_no': 2,
  'title': 'The Pony Remark',
  'air_date': '01/30/1991',
  'chrono_no': '7',
  'quotes': []},
 {'season_no': 2,
  'episode_no': 3,
  'title': 'The Jacket',
  'air_date': '02/0

In [7]:
class Episode:
    chrono_no = 0

    def __init__(self, season_no:int, ep_no:int,
                 title:str, air_date:str):
        self.season_no = season_no
        self.ep_no = ep_no
        self.title = title
        self.air_date = air_date
        Episode.chrono_no += 1

        zeroes = 3 - len(str(self.chrono_no)) 
        self.episode_id = f"E{zeroes*'0'}{self.chrono_no}{self.season_no}{self.ep_no}"


In [8]:
air_dates = []
for i in range(len(season_info_json)):
    air_dates.append(season_info_json[i]['air_dates'])

season_1 = Season(1, air_dates[0])
season_2 = Season(2, air_dates[1])
season_3 = Season(3, air_dates[2])
season_4 = Season(4, air_dates[3])
season_5 = Season(5, air_dates[4])
season_6 = Season(6, air_dates[5])
season_7 = Season(7, air_dates[6])
season_8 = Season(8, air_dates[7])
season_9 = Season(9, air_dates[8])

eps = episode_info_json


for e in eps:
    if e['season_no'] == 1:
        season_1.add_episode(e)
    elif e['season_no'] == 2:
        season_2.add_episode(e)
    elif e['season_no'] == 3:
        season_3.add_episode(e)
    elif e['season_no'] == 4:
        season_4.add_episode(e)
    elif e['season_no'] == 5:
        season_5.add_episode(e)
    elif e['season_no'] == 6:
        season_6.add_episode(e)
    elif e['season_no'] == 7:
        season_7.add_episode(e)
    elif e['season_no'] == 8:
        season_8.add_episode(e)
    elif e['season_no'] == 9:
        season_9.add_episode(e)

season_9.episodes

[{'season_no': 9,
  'episode_no': 1,
  'title': 'The Butter Shave',
  'air_date': '09/25/1997',
  'chrono_no': '157',
  'quotes': []},
 {'season_no': 9,
  'episode_no': 2,
  'title': 'The Voice',
  'air_date': '10/02/1997',
  'chrono_no': '158',
  'quotes': []},
 {'season_no': 9,
  'episode_no': 3,
  'title': 'The Serenity Now',
  'air_date': '10/09/1997',
  'chrono_no': '159',
  'quotes': []},
 {'season_no': 9,
  'episode_no': 4,
  'title': 'The Blood',
  'air_date': '10/16/1997',
  'chrono_no': '160',
  'quotes': []},
 {'season_no': 9,
  'episode_no': 5,
  'title': 'The Junk Mail',
  'air_date': '10/30/1997',
  'chrono_no': '161',
  'quotes': []},
 {'season_no': 9,
  'episode_no': 6,
  'title': 'The Merv Griffin Show',
  'air_date': '11/06/1997',
  'chrono_no': '162',
  'quotes': []},
 {'season_no': 9,
  'episode_no': 7,
  'title': 'The Slicer',
  'air_date': '11/13/1997',
  'chrono_no': '163',
  'quotes': []},
 {'season_no': 9,
  'episode_no': 8,
  'title': 'The Betrayal',
  'air_da

In [10]:
for e in episode_info_json:
    e['']

180